# Handwriting decoder evaluation

In [1]:
import os
import torch
from pytorch_lightning import Trainer
from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf

from generic_neuromotor_interface.data import make_handwriting_dataset
from generic_neuromotor_interface.handwriting_utils import CharacterErrorRates

TASK_NAME = "handwriting"
EMG_DATA_DIR = "~/emg_data"  # path to EMG data
MODELS_DIR = "~/emg_models"  # path to model files

/home/hh/miniconda3/envs/neuromotor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download Data Subset and Model Files

Before running this notebook, make sure you have downloaded the data (small subset) and model checkpoint.

You can do so with the following cell:

In [ ]:
# Equivalent to README instructions.

from generic_neuromotor_interface.scripts.download_data import download_data
from generic_neuromotor_interface.scripts.download_models import download_models

download_data(TASK_NAME, "small_subset", EMG_DATA_DIR)
download_models(TASK_NAME, MODELS_DIR)

In [2]:
if not os.path.exists(os.path.expanduser(EMG_DATA_DIR)):
    raise FileNotFoundError(f"The EMG data path does not exist: {EMG_DATA_DIR}")

if not os.path.exists(os.path.expanduser(MODELS_DIR)):
    raise FileNotFoundError(f"The models path does not exist: {MODELS_DIR}")

## Load model config

In [3]:
"""Load model config"""

config_path = os.path.join(os.path.expanduser(MODELS_DIR), TASK_NAME, "model_config.yaml")
config = OmegaConf.load(config_path)

## Load model checkpoint

In [5]:
"""Load model checkpoint"""
from generic_neuromotor_interface.lightning import HandwritingModule



model_ckpt_path = os.path.join(
    os.path.expanduser(MODELS_DIR),
    TASK_NAME,
    "model_checkpoint.ckpt"
)

model = HandwritingModule.load_from_checkpoint(
    model_ckpt_path,
    map_location=torch.device("cpu"),
)
# model = instantiate(config.lightning_module)
# model = model.load_from_checkpoint(
#     model_ckpt_path,
#     map_location=torch.device("cpu"),
# )

Lightning automatically upgraded your loaded checkpoint from v1.8.6 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../emg_models/handwriting/model_checkpoint.ckpt`


## Instantiate data module

In [6]:
"""Assemble the data module"""

# Update DataModule config with data path
config["data_module"]["data_location"] = os.path.expanduser(EMG_DATA_DIR)
if "from_csv" in config["data_module"]["data_split"]["_target_"]:
    config["data_module"]["data_split"]["csv_filename"] = os.path.join(
        os.path.expanduser(EMG_DATA_DIR),
        f"{TASK_NAME}_corpus.csv"
    )

datamodule = instantiate(config["data_module"])

## Run inference on one prompt

In [7]:
"""Grab one test prompt"""

test_dataset = make_handwriting_dataset(
    dataset_names=["handwriting_user_001_dataset_000"],
    data_location=datamodule.data_location,
    transform=datamodule.transform,
    padding=datamodule.padding,
    emg_augmentation=None,
    concatenate_prompts=False,
    min_duration_s=0.0,
)

sample = test_dataset[55]  # an arbitrary prompt from this dataset

[setup] Loading datasets for split None: 100%|██████████| 1/1 [00:00<00:00,  3.20it/s]


In [8]:
"""Run inference"""

model.eval()

# unpack sample
emg = sample["emg"]
labels = sample["prompts"]

# compute model outputs
with torch.no_grad():
    emissions, _slice = model(emg.T.unsqueeze(0))

    # compute greedy decode outputs
    predictions = model.decoder.decode_batch(
        emissions=emissions.movedim(0, 1).numpy(),
        emission_lengths=model.network.compute_time_downsampling(
            emg_lengths=torch.as_tensor([len(emg)]), slc=_slice
        )
    )

predictions = torch.as_tensor(predictions[0])

# convert predictions and labels to characters
predictions = model.decoder._charset.labels_to_str(predictions)
labels = model.decoder._charset.labels_to_str(labels)

In [9]:
"""Evaluate CER on this prompt"""

metric = CharacterErrorRates()
metric.update(
    prediction=predictions,
    target=labels,
)
aggregate_metrics = metric.compute()

print("CER of above prompt decode:", aggregate_metrics["CER"])

CER of above prompt decode: 21.428571428571427


In [10]:
"""Print predictions and target"""

print(
    f"Prediction: \t {predictions} \n"
    f"Target: \t {labels}"
)

Prediction: 	 alris ht thanhs 
Target: 	 alright thanks


## Evaluate full test set

Note that this requires you to have downloaded the full dataset (uncomment the below lines).

In [ ]:
## Uncomment to download the full dataset

# from generic_neuromotor_interface.scripts.download_data import download_data
# download_data(TASK_NAME, "full_data", EMG_DATA_DIR)

In [11]:
trainer = Trainer(accelerator="cpu")
test_results = trainer.test(model=model, datamodule=datamodule)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/hh/miniconda3/envs/neuromotor/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
[setup] Loading datasets for split test: 100%|██████████| 54/54 [00:07<00:00,  7.29it/s]
/home/hh/miniconda3/envs/neuromotor/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 6419/6419 [38:07<00:00,  2.81it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/CER            30.062541961669922
        test/DER            2.7663071155548096
        test/IER             8.627671241760254
        test/SER            18.668563842773438
        test_loss           1.3161078691482544
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
